In [1]:
import torch

In [2]:
!pip install git+https://github.com/huggingface/transformers accelerate bitsandbytes langchain sentence-transformers faiss-cpu

  Cloning https://github.com/huggingface/transformers to /private/var/folders/cs/hh3_26_57sx7l8sdk574zbdh0000gn/T/pip-req-build-4a461brl
  Running command git clone -q https://github.com/huggingface/transformers /private/var/folders/cs/hh3_26_57sx7l8sdk574zbdh0000gn/T/pip-req-build-4a461brl
  Resolved https://github.com/huggingface/transformers to commit 00c1d87a7d5c8dfb4554370983b5a3f7c069edd7
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 290 kB 8.8 MB/s eta 0:00:01
     |████████████████████████████████| 105.0 MB 86.3 MB/s eta 0:00:01
     |████████████████████████████████| 809 kB 42.0 MB/s eta 0:00:01
     |████████████████████████████████| 156 kB 37.1 MB/s eta 0:00:01
     |████████████████████████████████| 3.1 MB 99.7 MB/s eta 0:00:01
  Using cached scipy-1.12.0-cp310-cp310-macosx_12_0_arm64.whl (31.4 MB)
  Using cached SQLAlchemy-2.0.28-cp310-cp310-macosx_11_0_a

In [3]:
import transformers
from langchain.llms import HuggingFacePipeline
from langchain.chains import LLMChain, RetrievalQA, ConversationalRetrievalChain
from langchain.prompts import PromptTemplate
from langchain.vectorstores.faiss import FAISS
from langchain_core.documents import Document
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
import pandas as pd
from langchain_community.embeddings.openai import OpenAIEmbeddings
device = 'cuda' if torch.cuda.is_available() else 'cpu'

/Users/SamuelLP/Desktop/git/projet_nlp/.nlp_venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
model = transformers.AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")# , device_map='auto')
tokenizer = transformers.AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")

Loading checkpoint shards: 100%|██████████| 2/2 [01:09<00:00, 34.93s/it]
tokenizer_config.json: 100%|██████████| 1.47k/1.47k [00:00<00:00, 9.45MB/s]
tokenizer.model: 100%|██████████| 493k/493k [00:00<00:00, 17.9MB/s]
tokenizer.json: 100%|██████████| 1.80M/1.80M [00:00<00:00, 3.50MB/s]
special_tokens_map.json: 100%|██████████| 72.0/72.0 [00:00<00:00, 247kB/s]


In [10]:
import pandas as pd

train = pd.read_csv("../datas/train_data.csv")
test = pd.read_csv("../datas/test_data.csv")

In [11]:
def remove_newlines(df):
    df = df.replace("\n", '', regex=True)
    return df
train = remove_newlines(train)
test = remove_newlines(test)

In [12]:
train.head()

,ID,filename,texte,sexe,date_accident,date_consolidation
0,0,Agen_100515.txt,Le : 12/11/2019 Cour d’appel d’Agen chambre ...,homme,1991-04-09,n.c.
1,1,Agen_1100752.txt,Le : 12/11/2019 Cour d’appel d’Agen chambre ...,homme,2005-06-10,2010-01-19
2,2,Agen_1613.txt,Le : 12/11/2019 Cour d’appel d’Agen Audience...,femme,1997-09-26,n.c.
3,3,Agen_2118.txt,Le : 12/11/2019 Cour d’appel d’Agen Audience...,femme,1982-08-07,1982-11-07
4,4,Agen_21229.txt,Le : 12/11/2019 Cour d’appel d’Agen Audience...,homme,1996-11-26,n.c.


In [13]:
texte = train["texte"].tolist()

# RAG pipeline

In [14]:
import nltk
from nltk.tokenize import sent_tokenize

nltk.download('punkt') # Décommente cette ligne si 'punkt' n'a pas déjà été téléchargé

def preprocess_documents(df_or_series):
    pages_content = []
    
    # Vérifie si l'objet est une Series et le convertit en DataFrame si nécessaire
    if isinstance(df_or_series, pd.Series):
        df = df_or_series.to_frame().transpose()
    else:
        df = df_or_series
    
    for index, row in df.iterrows():
        texte = row["texte"]
        name = row["filename"]
        
        # Découpe le texte en phrases
        phrases = sent_tokenize(texte)
        
        for phrase in phrases:
            texte_doc = Document(page_content=phrase,
                                 metadata={'name': name})
            pages_content.append(texte_doc)
        
    return pages_content

[nltk_data] Downloading package punkt to /Users/SamuelLP/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [15]:
content = preprocess_documents(train.iloc[3,:])

In [16]:
embedding_model = SentenceTransformerEmbeddings(model_name='multi-qa-distilbert-cos-v1')

config_sentence_transformers.json: 100%|██████████| 116/116 [00:00<00:00, 449kB/s]
README.md: 100%|██████████| 9.52k/9.52k [00:00<00:00, 17.4MB/s]
sentence_bert_config.json: 100%|██████████| 53.0/53.0 [00:00<00:00, 251kB/s]
config.json: 100%|██████████| 523/523 [00:00<00:00, 3.47MB/s]
pytorch_model.bin: 100%|██████████| 265M/265M [00:03<00:00, 76.4MB/s] 
tokenizer_config.json: 100%|██████████| 333/333 [00:00<00:00, 2.18MB/s]
vocab.txt: 100%|██████████| 232k/232k [00:00<00:00, 2.94MB/s]
tokenizer.json: 100%|██████████| 466k/466k [00:00<00:00, 80.3MB/s]
1_Pooling/config.json: 100%|██████████| 190/190 [00:00<00:00, 857kB/s]


In [17]:
faiss_index = FAISS.from_documents(content, embedding_model)

In [18]:
retriever = faiss_index.as_retriever(search_kwargs={'k': 6})

In [19]:
text_generation_pipeline = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0,
    repetition_penalty=1.2,
    return_full_text=True,
    max_new_tokens=2000)

In [20]:
mistral_llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

In [21]:
qa = ConversationalRetrievalChain.from_llm(llm=mistral_llm,
                                           retriever=retriever,
                                           return_source_documents=True,
                                           verbose=False)

In [32]:
query = "Quelle est la date de consolidation?"
queries = ["Quel est le sexe de la victime?", "Quel est la date de l'accident?", "Quelle est la date de consolidation?"]

In [33]:
result = qa({"question" : f"""Tu es un assistant juridique. Tu vas recevoir plusieurs textes juridiques et tu vas devoir répondre à la recherché une ou des informations demandée(s) par l'utilisateur à partir des documents données.
On va te demander les informations suivantes : Sexe de la victime, date de l'accident, date de consolidation.
Le sexe de la victime est toujours contenu dans le document et ne peux prendre que deux valeurs : "homme" ou "femme". La classification du sexe de la victime a été réalisée en comptant le nombre de certains mots-clés comme dans le texte 'il' vs 'elle', 'monsieur' vs 'madame', 'né' vs 'née'...
La date de l'accident correspond à la date à laquelle l'accident s'est produit. Elle est toujours quelque part dans le document (généralement au début) sauf dans de très rares cas.. Nous attendons une date au format jj/mm/aaaa.
La date de consolidation est la date à laquelle les blessures de la victime sont devenues stables et ont été déclarées définitives par un médecin. L'information devrait être présente dans la plupart des cas mais parfois elle est soit manquante (tu afficheras la valeur "n.c.") soit non applicable (tu afficheras la valeur "n.a.") si la blessure n'a pas stabilisé avant le décès de la victime.
voici la demande : {query} """,
"chat_history": []})

/Users/SamuelLP/Desktop/git/projet_nlp/.nlp_venv/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [36]:
for element in result["source_documents"]:
    source = f'\n- {element.metadata["name"]}'
    print(source)


- Agen_2118.txt

- Agen_2118.txt

- Agen_2118.txt

- Agen_2118.txt

- Agen_2118.txt

- Agen_2118.txt


In [37]:
print(result['answer'])

 The information about the date of consolidation is not provided in the given texts.


In [38]:
source

'\n- Agen_2118.txt'